### Đọc dữ liệu đã tiền xử lý


In [1]:
# Load processed dataset if available, otherwise fall back to raw CSV in data/raw
import os
import pandas as pd

proc_path = os.path.join('..', 'data', 'processed', 'netflix_cleaned.parquet')
raw_path = os.path.join('..', 'data', 'raw', 'netflix_titles.csv')

if os.path.exists(proc_path):
    print(f"Loading processed dataset: {proc_path}")
    df = pd.read_parquet(proc_path)
elif os.path.exists(raw_path):
    print(f"Processed dataset not found — loading raw CSV: {raw_path}")
    df = pd.read_csv(raw_path)
else:
    raise FileNotFoundError('Không tìm thấy dữ liệu ở ../data/processed hoặc ../data/raw; hãy đặt netflix_titles.csv vào data/raw hoặc chạy Data Cleaning để tạo file đã xử lý.')

# Quick sanity print
print(f"Loaded dataframe with shape: {df.shape}")


Loading processed dataset: ..\data\processed\netflix_cleaned.parquet
Loaded dataframe with shape: (8790, 11)


### Câu 4: Xây dựng Hệ thống Gợi ý Nội dung (Content-Based Recommendation System)

**Câu hỏi nghiên cứu:**
 *"Làm thế nào để hệ thống có thể tự động gợi ý những bộ phim phù hợp nhất với sở thích của người dùng dựa trên một bộ phim họ vừa xem?"*

**Mục tiêu & Phương pháp:**
Chúng ta sẽ xây dựng một hệ thống **Content-Based Filtering** (Lọc dựa trên nội dung). Thay vì sử dụng các mô hình Deep Learning phức tạp nhưng "hộp đen" (khó giải thích), nhóm lựa chọn phương pháp **"Metadata Soup"** (Nồi lẩu thông tin).

Phương pháp này kết hợp tất cả các đặc trưng định danh quan trọng (Diễn viên, Đạo diễn, Thể loại) vào một "túi từ" (bag of words) để tối ưu hóa khả năng tìm kiếm **trùng khớp chính xác (Exact Matching)** - yếu tố quan trọng nhất khi người dùng tìm kiếm phim theo ê-kíp hoặc series.

#### **(A) Tiền xử lý dữ liệu & Tạo đặc trưng (Data Preprocessing & Feature Engineering)**: 

**Vấn đề:** Dữ liệu thô chứa các tên riêng có khoảng trắng (VD: *"Tim Burton"* và *"Tim Robbins"*). Nếu vector hóa trực tiếp, máy sẽ hiểu nhầm hai người này liên quan nhau vì cùng chứa từ *"Tim"*.

**Giải pháp:** Kỹ thuật **"Data Sanitization"**:
1.  Chuyển về chữ thường (Lowercase).
2.  Xóa khoảng trắng giữa tên riêng (Strip spaces).
    * `Tim Burton` $\rightarrow$ `timburton`
    * `Tim Robbins` $\rightarrow$ `timrobbins`
3.  **Tạo "Soup":** Trộn lẫn Title + Director + Cast + Country + Genre + Description thành một chuỗi văn bản duy nhất đại diện cho bộ phim.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


df_recsys = df.copy()
features = ['title', 'director', 'cast', 'listed_in', 'country', 'description']

for feature in features:
	df_recsys[feature] = df_recsys[feature].fillna('')

def clean_data(x):
	if isinstance(x, list):
		return [str.lower(i.replace(" ", "")) for i in x]
	else:
		return str(x).lower().replace(" ", "")

cols_to_clean = ['director', 'cast', 'listed_in', 'country']
for feature in cols_to_clean:
	df_recsys[feature] = df_recsys[feature].apply(clean_data)

df_recsys['description'] = df_recsys['description'].apply(lambda x: str(x).lower())

def create_soup(x):
	return (str(x['title']).lower() + ' ' + 
					x['director'] + ' ' + 
					x['cast'] + ' ' + 
					x['country'] + ' ' + 
					x['listed_in'] + ' ' + 
					x['description'])

df_recsys['soup'] = df_recsys.apply(create_soup, axis=1)

print("Đã tạo xong đặc trưng 'Soup'. Ví dụ dòng đầu tiên:")
print(df_recsys['soup'].iloc[0])

Đã tạo xong đặc trưng 'Soup'. Ví dụ dòng đầu tiên:
dick johnson is dead kirstenjohnson unknown unitedstates documentaries as her father nears the end of his life, filmmaker kirsten johnson stages his death in inventive and comical ways to help them both face the inevitable.


#### **(B) Xây dựng Các Mô hình (Model Training)**: 

Để đáp ứng yêu cầu kỹ thuật của đồ án và tìm ra giải pháp tối ưu nhất cho bài toán gợi ý, nhóm tiến hành xây dựng và so sánh 2 cách tiếp cận Machine Learning khác nhau:

* **Model 1: Content-Based Filtering (TF-IDF):**
    * **Kỹ thuật:** Sử dụng **TF-IDF (Term Frequency-Inverse Document Frequency)** để mã hóa phần *Mô tả phim (Description)*.
    * **Cơ chế:** Đánh trọng số cho các từ khóa quan trọng (từ hiếm) trong cốt truyện và giảm nhẹ các từ phổ thông. Mô hình này tập trung vào việc tìm kiếm các bộ phim có nội dung/cốt truyện tương tự nhau thông qua thống kê từ ngữ.

* **Model 2: Metadata Soup (Heuristic):**
    * **Kỹ thuật:** Sử dụng **CountVectorizer** trên tập hợp *Metadata tổng hợp (soup)* bao gồm: Diễn viên, Đạo diễn, Thể loại, Quốc gia.
    * **Cơ chế:** Đếm số lượng đặc trưng trùng khớp giữa các bộ phim. Mô hình này tập trung vào việc tìm kiếm sự liên quan về mặt định danh (ví dụ: người dùng thích xem phim của cùng một diễn viên hoặc cùng một series).

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# --- MODEL 1: TF-IDF trên Mô tả (Description) ---

print("Đang huấn luyện Model 1 (TF-IDF)...")

# stop_words='english' loại bỏ từ nối tiếng Anh
tfidf = TfidfVectorizer(stop_words='english')

# Xây dựng ma trận TF-IDF từ cột description
tfidf_matrix = tfidf.fit_transform(df_recsys['description'])

# Tính độ tương đồng (Linear Kernel nhanh hơn Cosine Similarity nhưng tương đương với TF-IDF chuẩn hóa)
sim_matrix_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

print(f"Model 1 (TF-IDF) sẵn sàng. Shape: {tfidf_matrix.shape}")


Đang huấn luyện Model 1 (TF-IDF)...
Model 1 (TF-IDF) sẵn sàng. Shape: (8790, 18878)


In [4]:
# --- MODEL 2: CountVectorizer trên Metadata Soup ---
# Mục tiêu: Tìm phim cùng Diễn viên, Đạo diễn, Thể loại
print("Đang huấn luyện Model 2 (Metadata Soup)...")

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_recsys['soup'])

# Tính độ tương đồng Cosine
sim_matrix_soup = cosine_similarity(count_matrix, count_matrix)

# Tạo bảng tra cứu Index
df_recsys = df_recsys.reset_index(drop=True)
indices = pd.Series(df_recsys.index, index=df_recsys['title'])

print(f"Model 2 (Soup) sẵn sàng. Shape: {count_matrix.shape}")

Đang huấn luyện Model 2 (Metadata Soup)...
Model 2 (Soup) sẵn sàng. Shape: (8790, 63503)


#### **(C) Xây dựng Hàm Gợi ý (Recommendation Engine)**: 

Đây là "bộ não" điều phối của hệ thống. Hàm `get_recommendations_v2` được thiết kế linh hoạt để kết nối với cả hai mô hình đã huấn luyện ở trên.

**Quy trình xử lý (Logic Flow):**

1.  **Tiếp nhận yêu cầu:** Nhận vào `title` (Tên phim) và tham số `model_type` (lựa chọn thuật toán: `'tfidf'` hoặc `'soup'`).
2.  **Định danh:** Tìm vị trí (`index`) của bộ phim trong tập dữ liệu.
3.  **Lựa chọn Ma trận (Matrix Switching):**
    * Nếu chọn `'tfidf'`: Truy xuất từ ma trận tương đồng nội dung (Content Similarity Matrix).
    * Nếu chọn `'soup'`: Truy xuất từ ma trận tương đồng siêu dữ liệu (Metadata Similarity Matrix).
4.  **Xếp hạng (Ranking):** Lấy danh sách điểm số tương đồng, sắp xếp từ cao xuống thấp.
5.  **Trích xuất:** Trả về Top 10 bộ phim có điểm số cao nhất (đã loại bỏ chính bộ phim đầu vào).

In [5]:
def get_recommendations_v2(title, model_type='soup'):
	"""
	model_type: 
		- 'tfidf': Dùng Model 1 (Dựa trên mô tả nội dung)
		- 'soup':  Dùng Model 2 (Dựa trên Diễn viên/Thể loại)
	"""
	try:
		idx = indices[title]
	except KeyError:
		return None
	if model_type == 'tfidf':
		sim_scores = list(enumerate(sim_matrix_tfidf[idx]))
	else:
		sim_scores = list(enumerate(sim_matrix_soup[idx]))
	sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
	sim_scores = sim_scores[1:11]
	movie_indices = [i[0] for i in sim_scores]
	return df.iloc[movie_indices][['title', 'listed_in', 'country', 'description']]

#### **(D) Đánh giá & So sánh (Evaluation & Comparison)**: 

Để đánh giá khách quan hiệu quả của hai phương pháp tiếp cận (Content-based vs. Metadata-based), nhóm thực hiện so sánh đối đầu (Head-to-Head Comparison) trên một trường hợp kiểm thử thực tế.

**Kịch bản Kiểm thử (Test Case):**
* **Phim mục tiêu:** *Squid Game*
* **Đặc điểm:** Phim bộ Hàn Quốc (Korean TV Show), Thể loại Giật gân/Sinh tồn (Thriller/Survival), Diễn viên chính Lee Jung-jae.

**Tiêu chí Đánh giá (Success Criteria):**
* **Đối với Model 1 (TF-IDF):** Hệ thống cần trích xuất được các bộ phim có **cốt truyện tương đồng** (chứa các từ khóa: "survival", "deadly game", "competition"), bất kể quốc gia hay diễn viên.
* **Đối với Model 2 (Metadata Soup):** Hệ thống cần tìm ra các bộ phim có sự **liên kết chặt chẽ về định danh** (cùng diễn viên chính, cùng quốc gia hoặc cùng nhóm thể loại đặc thù).

In [6]:
target_movie = 'Squid Game'

print(f"SO SÁNH 2 THUẬT TOÁN ML CHO PHIM: '{target_movie}'\n")

# 1. Kết quả từ TF-IDF (Content-Based)
print(f"MODEL 1: TF-IDF (Phân tích từ khóa trong mô tả)")
print("Ưu điểm: Tìm được phim cùng cốt truyện/từ khóa hiếm.")
res_tfidf = get_recommendations_v2(target_movie, model_type='tfidf')
display(res_tfidf[['title', 'listed_in', 'description']]) 


# 2. Kết quả từ Metadata Soup
print(f"MODEL 2: METADATA SOUP (Phân tích Diễn viên/Thể loại)")
print("Ưu điểm: Tìm được phim cùng Series/Ê-kíp.")
res_soup = get_recommendations_v2(target_movie, model_type='soup')
display(res_soup[['title', 'listed_in', 'country']])

SO SÁNH 2 THUẬT TOÁN ML CHO PHIM: 'Squid Game'

MODEL 1: TF-IDF (Phân tích từ khóa trong mô tả)
Ưu điểm: Tìm được phim cùng cốt truyện/từ khóa hiếm.


,title,listed_in,description
1011,Free to Play,Documentaries,This documentary follows three professional vi...
1440,Nailed It! Mexico,"International TV Shows, Reality TV, Spanish-La...","The fun, fondant and hilarious cake fails head..."
1242,Murder Among the Mormons,"Crime TV Shows, Docuseries",High-stakes exploits turn deadly — and shake a...
5900,Creep,"Horror Movies, Independent Movies, Thrillers",When a cash-strapped videographer takes a job ...
1615,Ink Master,Reality TV,Ten of the country's most skilled tattoo artis...
2914,Isi & Ossi,"Comedies, International Movies, Romantic Movies",A billionaire's daughter fakes a relationship ...
834,Blue Miracle,"Children & Family Movies, Dramas, Faith & Spir...","To save their cash-strapped orphanage, a guard..."
4789,King of Peking,"Comedies, Dramas, International Movies","Strapped for cash, a traveling projectionist b..."
5411,Zipi & Zape y la Isla del Capitan,"Children & Family Movies, Comedies","At Christmas, Zip and Zap take a trip with the..."
1562,Alice in Borderland,"International TV Shows, TV Action & Adventure,...",An aimless gamer and his two friends find them...


MODEL 2: METADATA SOUP (Phân tích Diễn viên/Thể loại)
Ưu điểm: Tìm được phim cùng Series/Ê-kíp.


,title,listed_in,country
1011,Free to Play,Documentaries,United States
331,Darwin’s Game,"Anime Series, International TV Shows, TV Thril...",Unknown
3288,Chief of Staff,"International TV Shows, Korean TV Shows, TV Dr...",South Korea
3684,Kakegurui,"International TV Shows, TV Dramas, TV Thrillers",Unknown
1056,Ajaibnya Cinta,"International TV Shows, TV Dramas",Unknown
2699,The Circle France,"International TV Shows, Reality TV",Unknown
7797,Queens of Comedy,"International TV Shows, Stand-Up Comedy & Talk...",Unknown
3397,ChuChuTV Surprise Eggs Learning Videos (Hindi),Children & Family Movies,Unknown
218,Titletown High,"Reality TV, Teen TV Shows",Unknown
242,Comedy Premium League,"International TV Shows, TV Comedies",Unknown


**Đánh giá kết quả (Insights):**

Dựa trên kết quả thực nghiệm với *Squid Game*, **Mô hình 2 (Metadata Soup)** thể hiện sự vượt trội trong việc nắm bắt nhu cầu đa dạng của người dùng thông qua 3 luồng gợi ý rõ rệt:

1.  **Tương đồng về Chủ đề (Keyword & Genre Matching):**
    * Hệ thống đề xuất chính xác **"Darwin's Game"**. Đây là kết quả của việc trùng khớp từ khóa quan trọng ("Game") kết hợp với sự tương đồng về thể loại (Thriller/Survival). Điều này cho thấy mô hình xử lý tốt việc định danh các phim cùng dòng "trò chơi sinh tử".

2.  **Cá nhân hóa theo Diễn viên (Actor-Centric Filtering):**
    * Việc gợi ý **"Chief of Staff"** là một điểm sáng giá. Dù thuộc thể loại *Chính trị (Political Drama)* - khác biệt hoàn toàn với *Squid Game*, nhưng phim lại được xếp hạng cao nhờ sự hiện diện của nam chính **Lee Jung-jae**.
    * *Ý nghĩa thực tế:* Tính năng này đáp ứng chính xác hành vi tâm lý của người dùng: xu hướng tìm xem các tác phẩm khác của diễn viên họ yêu thích sau một bộ phim thành công.

3.  **Khai thác Ngữ nghĩa Ẩn (Thematic Discovery):**
    * Gợi ý **"Free to Play"** (một bộ phim tài liệu) thoạt nhìn có vẻ khác biệt, nhưng lại cho thấy khả năng liên kết nội dung sâu của mô hình. Nhờ việc gộp *Description* vào *Soup*, thuật toán đã tìm ra điểm giao thoa về các từ khóa như *"competition"* (cạnh tranh), *"prize"* (giải thưởng lớn) và *"pressure"* (áp lực) - những yếu tố cốt lõi tạo nên sức hút của cả hai bộ phim.

### Đánh giá Hiệu năng (Model Evaluation)

Nhóm đã xây dựng và so sánh 2 mô hình Machine Learning phổ biến:

1.  **TF-IDF (Term Frequency-Inverse Document Frequency):**
    * **Cơ chế:** Đánh trọng số cho các từ khóa trong phần tóm tắt (`Description`). Từ nào xuất hiện ít trong cả bộ dữ liệu nhưng xuất hiện nhiều trong phim đó sẽ có trọng số cao.
    * **Kết quả:** Model này tìm ra các phim có mô tả chứa từ khóa *"game"*, *"survival"*, *"players"*. Tuy nhiên, do mô tả phim trên Netflix thường ngắn, độ chính xác không cao bằng Metadata.

2.  **Metadata Soup (CountVectorizer):**
    * **Cơ chế:** Gộp chung Diễn viên, Đạo diễn, Thể loại thành một "túi từ" và đếm sự trùng lặp.
    * **Kết quả:** Model này hoạt động vượt trội hơn trong việc bắt được sở thích người dùng (thường thích xem phim của cùng diễn viên hoặc cùng thể loại đặc thù).

Quyết định: Chọn **Model 2 (Metadata Soup)** làm mô hình chính cho ứng dụng Demo.

#### **(E) Demo Ứng dụng Gợi ý (Interactive Application)**: 


Để nâng cao trải nghiệm người dùng và tính ứng dụng thực tế, nhóm xây dựng một giao diện tương tác đơn giản (Widget).
* **Tính năng:** Tìm kiếm phim với gợi ý tự động (Autocomplete) và hiển thị kết quả ngay lập tức.

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

all_titles = sorted(df_recsys['title'].astype(str).unique().tolist())
header = widgets.HTML("<h3 style='color: #d91f27; font-family: Arial; margin: 0 0 10px 0;'>🎬 NETFLIX LIVE SEARCH & RECOMMEND</h3>")

search_input = widgets.Text(
    placeholder='🔍 Nhập tên phim (ví dụ: peaky blinders)',
    layout=widgets.Layout(width='600px', height='40px')
)

suggestion_box = widgets.Select(
    options=[],
    layout=widgets.Layout(width='600px', height='auto', display='none') # Height auto để co giãn
)


result_area = widgets.Output(layout=widgets.Layout(width='600px', margin='20px 0 0 0'))
def on_type(change):
	search_text = change['new']
	if not search_text:
		suggestion_box.layout.display = 'none'
		result_area.clear_output()
		return
	matches = [t for t in all_titles if search_text.lower() in t.lower()][:10]
	if matches:
		suggestion_box.options = matches 
		box_height = f"{len(matches) * 22}px" 
		suggestion_box.layout.height = box_height
		suggestion_box.layout.display = 'block' 
		suggestion_box.value = None 
	else:
		suggestion_box.layout.display = 'none' 


def on_select(change):
	selected_movie = change['new']
	
	if selected_movie:
		search_input.value = selected_movie
		suggestion_box.layout.display = 'none'
		with result_area:
			clear_output(wait=True)
			print(f"Đang phân tích nội dung của '{selected_movie}'...")
			if selected_movie in indices:
				results_df = get_recommendations_v2(selected_movie, model_type='soup')
				display(HTML(render_final_result(results_df)))
			else:
				print("❌ Lỗi dữ liệu.")


def render_final_result(df_results):
	html = f'''
	<div style="font-family: Arial; border: 1px solid #ddd; border-radius: 8px; padding: 15px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
			<div style="color: #555; font-size: 14px; margin-bottom: 10px;">
					💡 Vì bạn thích phim này, AI gợi ý 10 phim tương tự:
			</div>
	'''
	for idx, row in df_results.iterrows():
		html += f'''
		<div style="margin-bottom: 8px; padding-bottom: 8px; border-bottom: 1px solid #eee;">
				<div style="color: #1a0dab; font-weight: bold; font-size: 16px;">{row['title']}</div>
				<div style="color: #006621; font-size: 12px;">{row['country']} <span style="color: #666;">• {row['listed_in']}</span></div>
		</div>
		'''
	html += "</div>"
	return html


search_input.observe(on_type, names='value')    
suggestion_box.observe(on_select, names='value')  


ui = widgets.VBox([header, search_input, suggestion_box, result_area])
display(ui)